In [1]:
from konlpy.tag import Mecab
import numpy as np
import pandas as pd
from collections import Counter
from tensorflow.keras.preprocessing.sequence import pad_sequences
from gensim.models import Word2Vec
import matplotlib.pyplot as plt

train_data = pd.read_table('~/aiffel/sentiment_classification/data/ratings_train.txt')
test_data = pd.read_table('~/aiffel/sentiment_classification/data/ratings_test.txt')

tokenizer = Mecab()
stopwords = ['의', '가', '이', '은', '들', '는', '좀', '잘', '걍', '과', '도', '를', '으로', '자', '에', '와', '한', '하다']



In [2]:
def load_data(train_data, test_data, num_words=10000):
    # 데이터의 중복 제거
    train_data = list(set(train_data))
    test_data = list(set(test_data))
    
    # NaN 결측치 제거
    train_data = [data for data in train_data if str(data) != 'nan']
    test_data = [data for data in test_data if str(data) != 'nan']
    
    # 토큰화 및 불용어 제거
    X_train = [remove_stopwords(tokenizer.morphs(sentence)) for sentence in train_data]
    X_test = [remove_stopwords(tokenizer.morphs(sentence)) for sentence in test_data]
    
    # 단어 집합 생성
    words = np.concatenate(X_train).tolist()
    counter = Counter(words)
    if num_words is None:
        counter = counter.most_common(num_words)
    else:
        counter = counter.items()
    word_to_index = {word: index + 2 for index, (word, _) in enumerate(counter)}
    word_to_index['<PAD>'] = 0  # 패딩을 위한 토큰
    word_to_index['<UNK>'] = 1  # OOV(out-of-vocabulary)를 위한 토큰
    
    # 텍스트 스트링을 사전 인덱스 스트링으로 변환
    X_train = get_encoded_sentences(X_train, word_to_index)
    X_test = get_encoded_sentences(X_test, word_to_index)
    
    # 레이블 생성
    y_train = np.array([0] * len(X_train))
    y_test = np.array([0] * len(X_test))
    
    # 문장 길이 분포 확인
    total_data = X_train + X_test
    num_tokens = [len(tokens) for tokens in total_data]
    max_len = np.percentile(num_tokens, 95)  # 상위 95%의 문장 길이를 최대 문장 길이로 설정
    mean_len = np.mean(num_tokens)
    
    max_len = int(max_len)
    mean_len = int(mean_len)
    
    # 패딩 추가
    X_train = pad_sequences(X_train, maxlen=max_len, dtype=object)
    X_test = pad_sequences(X_test, maxlen=max_len, dtype=object)
    
    return X_train, y_train, X_test, y_test, word_to_index, max_len



In [3]:
def remove_stopwords(tokens):
    return [token for token in tokens if token not in stopwords]

def get_encoded_sentences(sentences, word_to_index):
    return [word_to_index['<PAD>']] + [word_to_index[word] if word in word_to_index else word_to_index['<UNK>'] for word in sentences]

# def get_encoded_sentences(sentences, word_to_index):
#     return [get_encoded_sentence(sentence, word_to_index) for sentence in sentences]

def plot_sentence_length_distribution(data):
    num_tokens = [len(tokens) for tokens in data]
    print(num_tokens)
    plt.hist(num_tokens, bins=100)
    plt.xlabel('Length of Sentences')
    plt.ylabel('Number of Sentences')
    plt.show()

# 예시 실행 코드
# train_data와 test_data는 각각 학습 데이터와 테스트 데이터로 구성된 리스트
X_train, y_train, X_test, y_test, word_to_index, max_len = load_data(train_data, test_data, num_words=10000)

# 문장 길이 분포 확인
plot_sentence_length_distribution(X_train)

# 모델 구성 및 validation set 구성
# 모델 훈련 개시
# Loss, Accuracy 그래프 시각화
# 학습된 Embedding 레이어 분석
# 한국어 Word2Vec 임베딩 활용하여 성능 개선


TypeError: unhashable type: 'list'

In [ ]:
plot_sentence_length_distribution(X_train)